In [1]:
import jax.numpy as np
from jax import random, grad, jit, vmap, vjp, lax
from jax import jacfwd, jacrev
import jax
jax.config.update("jax_enable_x64", True)
from jax.scipy.optimize import minimize

In [195]:
def func(x):
#     return (np.tanh(x)*np.cos(x))[0]
    return x*x

In [196]:
func_jit = jit(func)
func_jit(np.array([2.0]))

Array([4.], dtype=float64)

In [161]:
# res = minimize( func_jit, np.array([1.0]),np.array([2.0])), method='BFGS', tol=1e-8 ) 

In [162]:
res.x

Array([3.148954], dtype=float64)

In [163]:
# func_jit( res.x )

In [85]:
func_jit( np.array([1.0]), np.array([1.0]) )

Array(0.41149108, dtype=float64)

In [197]:
import torch
from jax2torch import jax2torch
torch_func_jit = jax2torch(func_jit)
class HorizonReward(torch.nn.Module):
    def __init__(self, size1):
        super().__init__()
        self.param = torch.nn.Parameter( torch.randn(size1), requires_grad=True )

    def forward(self):
        return torch_func_jit(self.param)
    
    def get_parameters(self):
        return self.param.item()

In [205]:
model = HorizonReward(1)

In [206]:
model.get_parameters()

-0.35270407795906067

In [207]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#1e-6)

In [208]:
for i in range(10000):
    cost = model()
    optimizer.zero_grad()
    cost.backward(retain_graph = False)
    optimizer.step()

In [209]:
model.get_parameters()

-3.9155544118657266e-39

In [210]:
model()

tensor([0.], grad_fn=<JaxFunBackward>)

In [211]:
model()

tensor([0.], grad_fn=<JaxFunBackward>)

In [201]:
0.299*0.299

0.089401

In [214]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms